In [1]:
import os
import sys
sys.path.append(os.pardir)

In [2]:
import json
import pathlib
import pandas as pd
from sqlalchemy.types import Integer, String, Boolean, DateTime
from credible import connectors

In [3]:
glob_pattern = '_x'
data_folderpath = os.path.join(os.pardir, 'data')
engine = connectors.connect_to_sqlite()

In [ ]:
def table_info(tablename, con):
    query = f'pragma table_info({tablename});'
    return pd.read_sql_query(sql=query, con=con)

def json_to_table(in_filepath, tablename, con):
    """ Puts json files into sql tables. """
    # read and write
    df = pd.read_json(os.path.join(data_folderpath, in_filepath))
    df.to_sql(name=tablename, con=con, if_exists='replace', index=False, chunksize=1000)
    
    # test
    out = pd.read_sql_query(sql=f'select * from {tablename}', con=con)
    assert df.equals(out)
    
    return out 

In [12]:
generator = pathlib.Path(data_folderpath).glob('*' + glob_pattern + '.json')

In [13]:
sorted_list = sorted(generator, key=os.path.getsize, reverse=True)
sorted_list

[PosixPath('../data/review_x.json'),
 PosixPath('../data/user_x.json'),
 PosixPath('../data/checkin_x.json'),
 PosixPath('../data/tip_x.json'),
 PosixPath('../data/business_x.json'),
 PosixPath('../data/photo_x.json')]

In [ ]:
for file in sorted_list:
    print(file.name)
    in_filepath = file.name
    tablename = file.stem
    con = engine
    
    json_to_table(in_filepath, tablename, con)